In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, SessionNotCreatedException
from time import sleep
from tqdm import tqdm
import pandas as pd

In [2]:
def get_product_content(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    section_elements = driver.find_elements(By.CLASS_NAME, class_name)
    for div in section_elements:
        p_tags = div.find_elements(By.TAG_NAME, 'p')
        for p in p_tags:
            yield p.text
    sleep(5)
    driver.quit()

In [3]:
def get_links_btn(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    div_elements = []
    for c in class_name:
        div_elements += driver.find_elements(By.CLASS_NAME, c)
    for div in div_elements:
        a_tag = div.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            yield a.get_attribute('href')
        # print(a_tag.text, a_tag.get_attribute('href'))
    sleep(2)
    driver.quit()

In [4]:
def get_num_last_page(link,ul_class='pagination_page_number'):
    driver = webdriver.Safari()
    driver.get(link)

    while (True):
        try:
            # Find the button element by its class
            load_more_button = driver.find_element(By.XPATH, '//a[@aria-label="Go to next page"]')
            load_more_button.click()
            print("Clicked 'Load More' button")
            sleep(5)
        except :
            ul = driver.find_element(By.CLASS_NAME, ul_class)
            li_elements = ul.find_elements(By.TAG_NAME, 'li')
            last_page = int(li_elements[-1].text.strip())
            break

    driver.quit()
    return last_page

In [5]:
def get_links_blogs(link,last_page):
    links = []
    for num_page in range(last_page):
        page_link = f"{link}/{num_page+1}"
        sleep(5)
        for l in get_links_btn(page_link,'blog_card_header'):
            links.append(l)
    return links

In [6]:
def check_exist_class(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    for name in class_name:
        try:
            element = driver.find_element(By.CLASS_NAME, name)
            if element:
                yield name
        except:
            pass

    sleep(2)
    driver.quit()

Footer

In [7]:
driver = webdriver.Safari()
driver.get("https://www.innovestx.co.th")

links = []
h3_element = driver.find_elements(By.CSS_SELECTOR, 'h3.head')

for h3 in h3_element:
    try:
        h3.click()
        print('click')
        div_elements = driver.find_element(By.CLASS_NAME, 'footer_section_one')
        a_tag = div_elements.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            links.append(a.get_attribute('href'))
        sleep(2)
    except:
        pass

driver.quit()

urls = list(set(links))
urls.sort()

click
click
click
click
click


In [14]:
links = urls[22:23]
links

['https://www.innovestx.co.th/products/Digital-asset/digital-asset']

In [15]:
visited_links = set()
href_class = ['blog_card_header','invest_guide ','swiper-wrapper','inner-plan-inverse', 'blog_list','container','pagination_page_number']
df = pd.DataFrame({'content':[],'link':[]})

while links:
    current_url = links.pop(0)
    content = " "
    
    if current_url in visited_links or current_url[:27] != "https://www.innovestx.co.th":
        continue
    print(current_url)
    if current_url[:32] == "https://www.innovestx.co.th/docs":
        new_record = ["Document", current_url]
        df.loc[len(df.index)] = new_record
        continue

    visited_links.add(current_url)

    exist_class = list(check_exist_class(current_url,href_class))
    
    # print(visited_links)
    # Block links
    if "pagination_page_number" in exist_class:
        try:
            last_page = get_num_last_page(current_url)
            links += get_links_blogs(current_url,last_page)
            print(last_page)

        except :
            # print('Num page not found')
            pass
    
    
    print(exist_class)
    sleep(5)
    try:
        links += list(get_links_btn(current_url,exist_class))
    except:
        pass
    
    if 'container' in exist_class:
        content = " ".join([text.strip() for text in get_product_content(current_url,'container')])

    links = list(set(links))
    new_record = [content, current_url]
    df.loc[len(df.index)] = new_record

df

https://www.innovestx.co.th/products/Digital-asset/digital-asset
['invest_guide ', 'swiper-wrapper', 'container']
https://www.innovestx.co.th/products/Digital-asset/digital-asset/Trading-Rule
['container']
https://www.innovestx.co.th/promotions/detail/SCB-Credit-Card-Rewards
['blog_card_header', 'swiper-wrapper', 'container']
https://www.innovestx.co.th/promotions/detail/PTT-Blue-Card-Rewards
['blog_card_header', 'swiper-wrapper', 'container']
https://www.innovestx.co.th/promotions/detail/SCB-Credit-Card-Rewards
https://www.innovestx.co.th/docs/default-source/da-document/da-off-20230820.pdf?sfvrsn=2f467f58_1
https://www.innovestx.co.th/products/product-user-guide/detail/manual/2565/09/29/DA-manual-04
['swiper-wrapper', 'container']
https://www.scb.co.th/th/personal-banking/tools/services-locator.html
https://www.innovestx.co.th/products/product-user-guide/detail/manual/2565/06/29/point-to-invest
['swiper-wrapper', 'container']
https://www.innovestx.co.th/search?indexCatalogue=sitesearc

https://www.innovestx.co.th/knowledge-hub/start-your-first-investment
https://www.innovestx.co.th/knowledge-hub/detail/economic-notes/invx-macro-making-sense-12062023


In [1]:
df

NameError: name 'df' is not defined